In [5]:
# Simulate a symbolic KB based on CIFAR-10 labels
# In practice, you would extract these from your dataset

cifar_labels = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

# Represent each image label as a fact: Object(label)
kb_facts = [f"Object({label})" for label in cifar_labels]

# Add rules for reasoning
kb_rules = [
    "Animal(X) :- Object(X), X in [cat, dog, bird, deer, frog, horse]",
    "Vehicle(X) :- Object(X), X in [airplane, automobile, ship, truck]"
]

# Combine facts and rules
kb = kb_facts + kb_rules


In [10]:
class BackwardChainer:
    def __init__(self, kb):
        self.facts = [fact for fact in kb if ":-" not in fact]
        self.rules = [rule for rule in kb if ":-" in rule]

    def solve(self, query):
        # Parse query predicate and argument
        pred, arg = query.split("(")
        arg = arg.rstrip(")")

        # Direct fact check
        if f"{pred}({arg})" in self.facts:
            return True

        # Rule processing
        for rule in self.rules:
            head, body = rule.split(":-")
            head_pred, head_arg = head.strip().split("(")
            head_arg = head_arg.rstrip(")")

            if head_pred == pred:
                # Split body into first predicate and remaining conditions
                body_parts = body.split(",", 1)  # Split on first comma only
                body_pred = body_parts[0].strip()

                # Parse body predicate and argument
                body_pred_name, body_pred_arg = body_pred.split("(")
                body_pred_arg = body_pred_arg.rstrip(")")

                # Check if body predicate matches a fact
                if f"{body_pred_name}({arg})" not in self.facts:
                    continue

                # Parse allowed list from remaining conditions
                if len(body_parts) > 1:
                    allowed_str = body_parts[1].split("[")[1].split("]")[0]
                    allowed = [x.strip() for x in allowed_str.split(",")]

                    if arg in allowed:
                        return True

        return False


In [11]:
def nl_to_symbolic(nl_query):
    # Simple mapping for demo
    mapping = {
        "Is cat an animal?": "Animal(cat)",
        "Is truck a vehicle?": "Vehicle(truck)",
        "Is dog an animal?": "Animal(dog)",
        "Is ship a vehicle?": "Vehicle(ship)"
    }
    return mapping.get(nl_query, "")


In [12]:
solver = BackwardChainer(kb)

# Example queries
nl_queries = [
    "Is cat an animal?",
    "Is truck a vehicle?",
    "Is frog an animal?",
    "Is airplane a vehicle?",
    "Is horse an animal?",
    "Is automobile an animal?"
]

for nl in nl_queries:
    symbolic = nl_to_symbolic(nl)
    if symbolic:
        result = solver.solve(symbolic)
        print(f"Q: {nl} → {symbolic} → {'Yes' if result else 'No'}")
    else:
        print(f"Q: {nl} → (No mapping)")


Q: Is cat an animal? → Animal(cat) → Yes
Q: Is truck a vehicle? → Vehicle(truck) → Yes
Q: Is frog an animal? → (No mapping)
Q: Is airplane a vehicle? → (No mapping)
Q: Is horse an animal? → (No mapping)
Q: Is automobile an animal? → (No mapping)
